In [0]:
dbutils.fs.ls("/mnt/bayerhackathon")

In [0]:
%sql
show databases;

In [0]:
%sql
create database if not exists hcl_hactathon_bronze;
create database if not exists hcl_hactathon_silver;
create database if not exists hcl_hactathon_gold;

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#reading customer.csv

In [0]:
#reading customer.csv

df_customer = spark.read.csv("/mnt/bayerhackathon/customer.csv", header=True, inferSchema=True)
df_customer.display()
df_customer.createOrReplaceTempView("customer_tbl")


In [0]:
%sql
select count(*) from customer_tbl

#reading customer_scd2.csv

In [0]:
#reading customer_scd2.csv

df_customer_SCD2_data = spark.read.csv("/mnt/bayerhackathon/customer_SCD2_data.csv", header=True, inferSchema=True,sep=";")
df_customer_SCD2_data.display()
df_customer_SCD2_data.createOrReplaceTempView("customer_SCD2_data_tbl")


In [0]:
%sql
select count(*) from customer_SCD2_data_tbl

In [0]:
df_customer_new = df_customer.join(df_customer_SCD2_data, df_customer.customer_id == df_customer_SCD2_data.customer_id, "left").select(df_customer_SCD2_data['*'],lit('Yes').alias('loyalty')).filter(df_customer_SCD2_data.customer_id.isNotNull()).distinct()
df_customer_new.display()
df_customer_new.createOrReplaceTempView("customer_new_tbl")

In [0]:
df_customer_updated = spark.sql("select *, 'Yes' as loyalty from customer_SCD2_data_tbl where customer_id not in (select distinct customer_id from customer_new_tbl)")

## Customer Data

In [0]:
df_customer_final = df_customer.union(df_customer_new).union(df_customer_updated)
# df_customer_final.filter(col("phone").isNotNull())
df_customer_final.createOrReplaceTempView("customer_final_tbl")
df_customer_final.write.mode("overwrite").format("delta").saveAsTable("hcl_hactathon_bronze.customer")

In [0]:
df_customer_final_null_removal = spark.sql("select * from hcl_hactathon_bronze.customer where phone is not null")
df_customer_final_null_removal.write.mode("overwrite").format("delta").saveAsTable("hcl_hactathon_silver.customer")

In [0]:
%sql
select count(*) from hcl_hactathon_bronze.customer

In [0]:
%sql
select count(*) from hcl_hactathon_silver.customer

In [0]:
%sql
select * from hcl_hactathon_silver.customer 
where phone is null

In [0]:
df_customer_behaviour.count()

In [0]:
#reading customer_SCD2_data.csv

df_customer_behaviour = spark.read.csv("/mnt/bayerhackathon/customer_behaviour.csv", header=True, inferSchema=True)
# df_customer_behaviour.display()
# df_customer_behaviour.count()
df_customer_behaviour.createOrReplaceTempView("customer_behaviour_tbl")
df_customer_behaviour.write.mode("overwrite").format("delta").saveAsTable("hcl_hactathon_bronze.customer_behaviour")

df_customer_behaviour_final = spark.sql("select * from customer_behaviour_tbl where customer_id not in (select distinct customer_id from hcl_hactathon_bronze.customer where phone is null)")
df_customer_behaviour_final.write.mode("overwrite").format("delta").saveAsTable("hcl_hactathon_silver.customer_behaviour")

In [0]:
%sql
select count(*) from hcl_hactathon_bronze.customer_behaviour

In [0]:
%sql
select count(*) from hcl_hactathon_silver.customer_behaviour

# reading order.csv

In [0]:
# reading order.csv
df_order = spark.read.csv("/mnt/bayerhackathon/order.csv", header=True, inferSchema=True)
# df_order.display()
# df_order.count()
df_order.createOrReplaceTempView("order_tbl")
df_order.write.mode("overwrite").format("delta").saveAsTable("hcl_hactathon_bronze.order")

df_order_final = spark.sql("select * from order_tbl where customer_id not in (select distinct customer_id from hcl_hactathon_bronze.customer where phone is null)")
df_order_final.write.mode("overwrite").format("delta").saveAsTable("hcl_hactathon_silver.order")


In [0]:
%sql
select count(*) from hcl_hactathon_bronze.order

In [0]:
%sql
select count(*) from hcl_hactathon_silver.order

# reading order_line.csv

In [0]:
# reading order_line.csv
df_order_line = spark.read.csv("/mnt/bayerhackathon/order_line.csv", header=True, inferSchema=True)
# df_order_line.display()
# df_order_line.count()
df_order_line.createOrReplaceTempView("order_line_tbl")
df_order_line.write.mode("overwrite").format("delta").saveAsTable("hcl_hactathon_bronze.order_line")

df_order_line_final = spark.sql("select * from order_line_tbl where order_id not in (select distinct order_id from hcl_hactathon_bronze.order where customer_id  in (select distinct customer_id from hcl_hactathon_bronze.customer where phone is null))")
df_order_line_final.write.mode("overwrite").format("delta").saveAsTable("hcl_hactathon_silver.order_line")


In [0]:
%sql
select count(*) from hcl_hactathon_bronze.order_line

In [0]:
%sql
select count(*) from hcl_hactathon_silver.order_line

## Gold layer

## Orders Datamart

In [0]:
df_order = spark.read.table("hcl_hactathon_silver.order").distinct()
df_order_line = spark.read.table("hcl_hactathon_silver.order_line").distinct()

df_order_fact = df_order.join(df_order_line, df_order.order_id == df_order_line.order_id, "inner").select(df_order['*'],df_order_line['order_line_id'],df_order_line['product'],df_order_line['quantity'],df_order_line['price'],df_order_line['order_currency'],(df_order_line['quantity']*df_order_line['price']).alias('updated_total_purchase_value'))
df_order_fact.display()

df_order_fact.write.mode("overwrite").format("delta").saveAsTable("hcl_hactathon_gold.order_datamart")


## Customer Datamart

In [0]:
df_customer = spark.read.table("hcl_hactathon_silver.customer").distinct()
df_customer_behaviour = spark.read.table("hcl_hactathon_silver.customer_behaviour").distinct()

df_customer_dimension = df_customer.join(df_customer_behaviour, df_customer.customer_id == df_customer_behaviour.customer_id, "inner").select(df_customer['*'],df_customer_behaviour['order_frequency'],df_customer_behaviour['average_order_value'],df_customer_behaviour['customer_lifetime_value'],df_customer_behaviour['website_visits'],df_customer_behaviour['seconds_spent_on_website'],df_customer_behaviour['page_views'],df_customer_behaviour['cart_abandonment_rate'])

df_customer_dimension.display()

df_customer_dimension.write.mode("overwrite").format("delta").saveAsTable("hcl_hactathon_gold.customer_datamart")


## Orders 360 View

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df_orders_360_view = df_order_fact.join(df_customer_dimension, (df_order_fact.customer_id == df_customer_dimension.customer_id), "left").select(df_order_fact['*'],df_customer['first_name'],df_customer['last_name'],df_customer['email'],df_customer['gender'],df_customer['Address'],df_customer['city'],df_customer['country'],df_customer['zipcode'],df_customer['phone'],df_customer['Created_date'],df_customer['loyalty'],df_customer_dimension['order_frequency'],df_customer_dimension['average_order_value'],df_customer_dimension['customer_lifetime_value'],df_customer_dimension['website_visits'],df_customer_dimension['seconds_spent_on_website'],df_customer_dimension['page_views'],df_customer_dimension['cart_abandonment_rate'], df_customer_dimension['state'].alias('customer_state'))

df_orders_360_view = df_orders_360_view.withColumn('address1',regexp_extract(df_orders_360_view.Address, '([0-9]+)', 1)).withColumn('address2',regexp_extract(df_orders_360_view.Address, '([^0-9]+)', 1))

df_orders_360_view.display()

df_orders_360_view.write.mode("overwrite").format("delta").saveAsTable("hcl_hactathon_gold.orders_360_view")

In [0]:
%sql
select count(*) from hcl_hactathon_gold.orders_360_view